<a href="https://colab.research.google.com/github/beeyeas/colab/blob/main/EmailNsights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib streamlit streamlit_js streamlit-google-auth streamlit_javascript

In [ ]:
# prompt: EmailInsights is an innovative analytics app designed to help individuals and businesses uncover hidden trends, patterns, and insights within their email inboxes. By connecting to popular email services (e.g., Gmail, Outlook, Yahoo), EmailInsights web application built using streamlit provides users with a comprehensive dashboard to:
# # 1. Track email metrics: Monitor email volume, response rates, and engagement metrics.
# # 2. Analyze sender and recipient patterns: Identify frequent contacts, most emailed topics, and communication networks.
# # 3. Detect email categorization and prioritization: Automatically categorize emails (e.g., work, personal, spam) and prioritize important messages.
# # 4. Visualize email data: Interactive charts, graphs, and heatmaps to help users understand their email behavior.
# # Core Features:
# # 1. Email account integration: Seamless connection to popular email services like Gmail using Gmail API
# # 2. Data visualization: Customizable dashboards with interactive charts and graphs.
# # 3. Machine learning-powered insights: Automated analysis and recommendations.
# # 4. Customizable alerts and notifications: Set alerts for important emails, sender/recipient patterns, and email volume.
# # use Gmail API client "pip install google-api-python-client"
%%writefile app.py

import streamlit as st
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) # You'll need credentials.json
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def get_email_data(service):
  # Placeholder for actual email data retrieval logic.
  # Replace with actual implementation using Gmail API.
  results = service.users().messages().list(userId='me', maxResults=10).execute()
  messages = results.get('messages', [])
  email_data = []
  for msg in messages:
    msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
    email_data.append(msg_data)
  return email_data

# Streamlit app
st.title("EmailInsights")

if st.button("Connect to Gmail"):
    creds = authenticate_gmail()
    if creds:
        service = build('gmail', 'v1', credentials=creds)
        email_data = get_email_data(service)
        st.write("Connected to Gmail successfully!")
        # Process and display data as needed using Streamlit elements
        # Example display of email snippets
        for msg in email_data:
            try:
              snippet = msg['snippet']
              st.write(snippet)
            except KeyError:
              st.write("No snippet found for this email")


    else:
        st.write("Authentication failed.")



Overwriting app.py


In [ ]:
%%writefile app.py

import os
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
import streamlit as st
import webbrowser

redirect_uri = os.environ.get("REDIRECT_URI", "http://localhost:8501/")

def auth_flow():
    st.write("Welcome to My App!")
    auth_code = st.query_params.get("code")
    flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
        "credentials.json", # replace with you json credentials from your google auth app
        scopes=["https://www.googleapis.com/auth/userinfo.email", "openid"],
        redirect_uri=redirect_uri,
    )
    if auth_code:
        flow.fetch_token(code=auth_code)
        credentials = flow.credentials
        st.write("Login Done")
        user_info_service = build(
            serviceName="oauth2",
            version="v2",
            credentials=credentials,
        )
        user_info = user_info_service.userinfo().get().execute()
        assert user_info.get("email"), "Email not found in infos"
        st.session_state["google_auth_code"] = auth_code
        st.session_state["user_info"] = user_info
    else:
        if st.button("Sign in with Google"):
            authorization_url, state = flow.authorization_url(
                access_type="offline",
                include_granted_scopes="true",
            )
            webbrowser.open_new_tab(authorization_url)


def main():
    if "google_auth_code" not in st.session_state:
        auth_flow()

    if "google_auth_code" in st.session_state:
        email = st.session_state["user_info"].get("email")
        st.write(f"Hello {email}")


if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
%%writefile app.py

import os
import json
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
import streamlit as st
import webbrowser

redirect_uri = os.environ.get("REDIRECT_URI", "https://www.google.com")
from streamlit_js import st_js, st_js_blocking

def ls_get(k, key=None):
    return st_js_blocking(f"return JSON.parse(localStorage.getItem('{k}'));", key)


def ls_set(k, v, key=None):
    jdata = json.dumps(v, ensure_ascii=False)
    st_js_blocking(f"localStorage.setItem('{k}', JSON.stringify({jdata}));", key)

def init_session():
    user_info = ls_get("user_info")
    if user_info:
        st.session_state["user_info"] = user_info

def auth_flow():
    st.write("Welcome to My App!")
    auth_code = st.query_params.get("code")

    flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
        "credentials.json", # replace with you json credentials from your google auth app
        scopes=["https://www.googleapis.com/auth/userinfo.email", "openid"],
        redirect_uri=redirect_uri,
    )
    if auth_code:
        flow.fetch_token(code=auth_code)
        credentials = flow.credentials
        st.write("Login Done")
        user_info_service = build(
            serviceName="oauth2",
            version="v2",
            credentials=credentials,
        )
        user_info = user_info_service.userinfo().get().execute()
        assert user_info.get("email"), "Email not found in infos"
        st.session_state["google_auth_code"] = auth_code
        st.session_state["user_info"] = user_info
        ls_set("user_info", user_info)
        # TODO fix calling consecutive ls_set is not working
        # ls_set("google_auth_code", auth_code)
    else:
        authorization_url, state = flow.authorization_url(
            access_type="offline",
            include_granted_scopes="true",
        )
        st.link_button("Sign in with Google", authorization_url)

def main():
    init_session()
    if "user_info" not in st.session_state:
        print("not in session proceeding to auth_flow")
        auth_flow()

    if "user_info" in st.session_state:
        #main_flow()
        print(st.session_state["user_info"])
        email = st.session_state["user_info"].get("email")
        st.write(f"Hello {email}")

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
# Trying OAUTH using streamlit-google-auth
%%writefile app.py

import streamlit as st
from streamlit_google_auth import Authenticate
import streamlit as st
from streamlit_javascript import st_javascript

#url = st_javascript("await fetch('').then(r => window.parent.location.href)")
url = st_javascript("await fetch('').then(r => window.parent.location.href)").replace(" ", "")
st.write(url)
print(url)
st.title('Streamlit Google Auth Example 2')

authenticator = Authenticate(
    secret_credentials_path = 'credentials.json',
    cookie_name='my_cookie_name',
    cookie_key='this_is_secret',
    redirect_uri = url,
)

# Catch the login event
authenticator.check_authentification()

if st.session_state['connected']:
    st.image(st.session_state['user_info'].get('picture'))
    st.write('Hello, '+ st.session_state['user_info'].get('name'))
    st.write('Your email is '+ st.session_state['user_info'].get('email'))
    if st.button('Log out'):
        authenticator.logout()
else:
    st.write('You are not connected')
    authorization_url = authenticator.get_authorization_url()
    st.markdown(f'[Login]({authorization_url})')
    st.link_button('Login', authorization_url)

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm